# Putting it all together

In [2]:
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig
from frame.frame import Frame

periods = 365

ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-1, 1),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

spy = RandomOHLCV( 
    freq      = '15 min', 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
).get_dataframe()

ohlcv['SPY_cl'] = spy['close'] # add spy close to ohlcv


f = Frame('TSLA')
f.load_ohlcv(ohlcv)
f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='line')
# f.add_ta(ta.MA('close', 9),  {'dash': 'solid', 'color': 'magenta', 'width': 2}) # appends ta to the ta list
# f.add_ta(ta.MA('close', 21), {'dash': 'solid', 'color': 'cyan', 'width': 1})
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2})
# f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2)
# f.add_ta(ta.MACD(fast=12, slow=26, signal=9) , 
#          [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
#           {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
#           {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type = 'points')

# f.add_ta(ta.SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
#             [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
#             {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
#             chart_type = 'support_resistance') 



f.add_ta(ta.ConsolidationZone(hp_column='HP_hi_10', lp_column='LP_lo_10', atr_column='ATR_50', price_tolerance=0.001, max_points_between=1, height_width_ratio=50),
            [{'color': "rgba(225, 182, 30, 0.5)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'color': "rgba(225, 182, 30, 0.1)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'cons')

# f.add_ta(ta.TrendlineDetector(name='TREND', point_column='LP_lo_10',slope_direction='up', slope_tolerance=0.5, min_points=3, lookback_points=6),
#             {'dash': 'solid', 'color': 'yellow', 'width': 2}, 
#             chart_type = 'trendlines')

# f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='HP_hi_10', atrCol='ATR_50', slopeDir='down', slopeToleranceATR=2, projectionPeriod=10),
#             {'dash': 'solid', 'color': 'rgb(173, 90, 255)', 'width': 1}, 
#             chart_type = 'trendlines')

# f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='LP_lo_10', atrCol='ATR_50', slopeDir='up', slopeToleranceATR=2, projectionPeriod=10),
#             {'dash': 'solid', 'color': 'rgb(198, 255, 90)', 'width': 1}, 
#             chart_type = 'trendlines')

# f.add_ta(sig.Tail(ls='LONG', lookBack=100, normRange=(0,100), tailExceedsNthBarsAgo=2), 
#               {'dash': 'solid', 'color': 'red', 'width': 1}, 
#                chart_type = 'lines+markers', row=4) # tail signal

# f.add_ta(sig.PullbackNear(ls='LONG', lookBack=100, normRange=(0,100), maxCol='HP_hi_10', minCol='LP_lo_10',  pullbackCol='MA_cl_21', optimalRetracement=99 ),
#               {'dash': 'solid', 'color': 'blue', 'width': 1}, 
#               chart_type = 'lines+markers', row=4) # pullback signal

# f.add_ta(sig.Overlap(ls='LONG', lookBack=100, normRange=(0,100), maxCol='HP_hi_10', minCol='LP_lo_10' ),
#               {'dash': 'solid', 'color': 'purple', 'width': 1}, 
#               chart_type = 'lines+markers', row=4) # overlap signal

#! Mansfield RSI
# f.add_ta(ta.MansfieldRSI(close_col='close', market_col='SPY_cl', span=14), 
#          {'dash': 'solid', 'color': 'yellow', 'width': 2}, 
#          row=3) # mansfield rsi signal

#$ ---- Consilidation Signals ----

colours = ['blue', 'yellow', 'green', 'red', 'orange', 'magenta', 'cyan', 'purple']

# f.add_ta(sig.CountTouches(ls='LONG', supOrRes='sup', normRange=(0,10), lookBack=periods-1, columnStartsWith='MTREND_UP', atrCol='ATR_50', touchTolerance=0.5),
#               [{'dash': 'solid', 'color': color, 'width': 1} for color in colours],
#               chart_type = 'lines+markers', row=4) # count touches signal

# f.add_ta(sig.LineLengths(normRange=(0,100), lookBack=periods-1, columnStartsWith='MTREND_UP', minBars=10),
#               [{'dash': 'solid', 'color': color, 'width': 1} for color in colours],
#               chart_type = 'lines+markers', row=4) # count touches signal

f.add_ta(sig.ConsolidationShape(normRange=(0,100), consUpperCol='CONS_UPPER', consLowerCol='CONS_LOWER', atrCol='ATR_50', minBars=10, lookBack=periods-1),
              [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
              chart_type = 'line', row=4) # count

f.add_ta(sig.ConsolidationPosition(normRange=(0,100), consUpperCol='CONS_UPPER', consLowerCol='CONS_LOWER', lookBack=periods-1),
              [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
              chart_type = 'line', row=4) # count

f.add_ta(sig.ConsolidationPreMove(normRange=(0,15), consUpperCol='CONS_UPPER', consLowerCol='CONS_LOWER', maCol='MA_cl_50', atrCol='ATR_50', lookBack=periods-1),
              [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
              chart_type = 'line', row=4) # count

#$---- Trendline Signals ----
# f.add_ta(sig.CountTouches(ls='LONG', supOrRes='sup', normRange=(0,10), lookBack=periods-1, columnStartsWith='MTREND_UP', atrCol='ATR_50', touchTolerance=0.5),
#               [{'dash': 'solid', 'color': color, 'width': 1} for color in colours],
#               chart_type = 'lines+markers', row=4) # count touches signal

# f.add_ta(sig.LineLengths(normRange=(0,100), lookBack=periods-1, columnStartsWith='MTREND_UP', minBars=10),
#               [{'dash': 'solid', 'color': color, 'width': 1} for color in colours],
#               chart_type = 'lines+markers', row=4) # count touches signal




#$ ---- Gapped signals----
# f.add_ta(sig.IsGappedOverPivot(ls='LONG', normRange=(0,1), pointCol='HP_hi_10',lookBack=100),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # gapped over pivot signal

# f.add_ta(sig.GappedPivots(ls='LONG', normRange=(0, 5), pointCol='HP_hi_10',  span=400, lookBack=100),
#               {'dash': 'solid', 'color': 'orange', 'width': 1}, 
#               chart_type = 'lines+markers', row=4) # gapped pivots signal

# f.add_ta(sig.GappedRetracement(ls='LONG', normRange=(0,200), pointCol='HP_hi_10', atrCol='ATR_50', lookBack=100),
#               {'dash': 'solid', 'color': 'magenta', 'width': 1},  
#               chart_type = 'lines+markers', row=4) # gapped bar quality signal


# f.add_ta(sig.GappedPastPivot(ls='LONG', normRange=(0,100), atrCol='ATR_50', pointCol='HP_hi_10', lookBack=100, maxAtrMultiple=10),
#               {'dash': 'solid', 'color': 'red', 'width': 1},
#               chart_type = 'lines+markers', row=4) # gapped past pivot signal

# f.add_ta(sig.GapSize(ls='LONG', normRange=(0,300), pointCol='HP_hi_10', atrCol='ATR_50', lookBack=100), # as pct of ATR
#               {'dash': 'solid', 'color': 'red', 'width': 1},
#               chart_type = 'lines+markers', row=4) # gap size signal
         

# f.add_ta(sig.Score( name = 'L_Gaps', normRange=(0,100), lookBack=100, cols = ['SigL_GPiv', 'SigL_GRtc', 'SigL_GPP', 'SigL_GSiz'], scoreType = 'sum', weight = 1),
#               {'dash': 'solid', 'color': 'yellow', 'width': 2},
#               chart_type = 'lines+markers', row=4) # score signal

#$ ---- Volume signals ----
# f.add_ta(sig.VolumeSpike(ls='LONG', normRange=(0,200), volMACol='MA_vo_10', lookBack=100),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # volume spike signal

# f.add_ta(sig.VolumeROC(ls='LONG', normRange=(0,300), lookBack=100),
#               {'dash': 'solid', 'color': 'red', 'width': 1},
#               chart_type = 'lines+markers', row=4) # volume roc signal

# f.add_ta(sig.Score( name = 'L_Vol', cols = ['SigL_VolSpike', 'SigL_VolROC'], scoreType = 'max', weight = 1, lookBack=100),
#               {'dash': 'solid', 'color': 'yellow', 'width': 2},
#               chart_type = 'lines+markers', row=4) # score signal

# ---- Room To Move signals ----
# f.add_ta(sig.RoomToMove(ls='LONG', normRange=(0,2), atrCol='ATR_50', tgetCol='Res_1_Lower', lookBack=100),
#               {'dash': 'solid', 'color': 'green', 'width': 1},
#               chart_type = 'lines+markers', row=4) # room to move signal


#$ ---- Utils and Ta----

# f.add_ta(sig.ExampleClass(name='Example', ls='LONG', normRange=(0,100), lookBack=100),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # example signal

# #! get HP or LP index

# # #£ get signal @ HP or LP
# f.add_ta(sig.GetSignalAtPoint(ls='LONG', normRange=(0,100), lookBack=100, pointCol='HP_hi_10', sigCol='SigL_Example', pointsAgo=1),
#               {'dash': 'solid', 'color': 'pink', 'width': 1},
#               chart_type = 'lines+markers', row=4) # example signal

# #£ get max signal since HP or LP (within nth bars after HP or LP)
# f.add_ta(sig.GetMaxSignalSincePoint(ls='LONG', normRange=(0,100), lookBack=100, pointCol='HP_hi_10', sigCol='SigL_Example', pointsAgo=1),
#               {'dash': 'solid', 'color': 'yellow', 'width': 1},
#               chart_type = 'lines+markers', row=4) # example signal


#! trendlines (from HP or LP)

#! consolidation (recent moves within Sup1 and Sup2 or Res1 and Res2)


#$ ---- Pullback Bounce signals ----

#£ recent hp low > current support
#£ since recent HP lowest point < current support
#£ close > support
# f.add_ta(sig.PullbackBounce(ls='LONG', normRange=(0,1), lookBack=100, pointCol='HP_hi_10', supResCol='Sup_1_Lower'),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # pullback bounce signal

#£ Volume spike on recnt HP > 80% of the MA10 volume

#£ retracment ideally 50% of the recent HP
# f.add_ta(sig.Trace(ls='LONG', normRange=(0,100), pointCol='HP_hi_10', atrCol='ATR_50', lookBack=100),

#£ lower highs on the retracement

#£ overlap on the retracement

#£ Retraces to the MA21
#£ Retraces to the MA50

#£ recent low point has long tail


#$ ---- BreakOut signals ----

#£ Crossed above resistance (bar low > resistance, prev bar lowerer than resistance)

#£ Volume spike on breakout > 80% of the MA10 volume

#! Resistance is significant (number of touches)

#! breaks consolidation


#$ ---- Prev Resistance Significance signals (now Support) ----

#! number of touches




f.setup_chart() 
f.plot(trading_hours=True, height=1000, width=1500)


"""
Resistance: 


The objective is to modify a support and resistance class that will find the support and resistance levels and return a pandas data frame with those levels projected forward to the end of the data frame from the points that have been detected
So it is important that every row of the data frame is filled in with the same price point stretching from the index where the point is located to the end of the data frame so that it displays correctly on a chart. (This is not a visualisation task but a data manipulation task)
Do not worry about tolerances and touch points for now.

  -- All code snippets are to be returned in the chat do not update files directly
  -- compute only base on the last close price on the last bar
  -- hp and lp is high point or low point 
  -- HP and LP are already computed and should be obtained from the data frame by referencing the relevant columns
  -- find the next  hp or lp That is higher than the close price. I suggest putting them all in a list both high points and low points and sorting them in ascending order first.
  -- When finding the next higher point use the argument pointsAgo to find the point that is x points higher from the current point.
  -- now find to index Of the point that was found whether it was a low point or a high point is irrelevant. 
  -- Now project the point that was found and forward fill it to the end of the df So that when visualised on a chart there will be a line stretching from the point that has been found to the end of the chart. 
  
  Support:
  -- Similar to resistance but of course the opposite.

  """

# name = f"{self.name}_{self.slopeDir.upper()}_{i+1}"
cols = [c for c in f.data.columns if 'CONS' in c]
f.data[cols].dropna(how='all')



7. Found 1 zones
pivot_time: 2024-01-02 18:00:00, pivot_price: 92.9478
crossover_time: None, crossover_price: None


,CONS_UPPER_1,CONS_LOWER_1
date,,
2024-01-02 23:30:00,94.65,92.79
2024-01-02 23:45:00,94.65,92.79
2024-01-03 00:00:00,94.65,92.79
2024-01-03 00:15:00,94.65,92.79
2024-01-03 00:30:00,94.65,92.79
...,...,...
2024-01-04 05:45:00,94.65,92.79
2024-01-04 06:00:00,94.65,92.79
2024-01-04 06:15:00,94.65,92.79


In [3]:

"""What is a touch? (if line is support)
1. the low is within tolerance above the line of the line
2. the high not below the line of the line
3. the high > the line and the low < the line (eg the line is within the bar)
4. there are more lows above the line than there are highs below the line since the last touch
"""

[(ATR(column=None, hi_col='high', lo_col='low', span=50), {}, None, 1),
 (MA(column='close', period=21),
  {'dash': 'solid', 'color': 'cyan', 'width': 1},
  'line',
  1),
 (MA(column='close', period=50),
  {'dash': 'solid', 'color': 'magenta', 'width': 2},
  'line',
  1),
 (MA(column='volume', period=10),
  {'dash': 'solid', 'color': 'yellow', 'width': 1},
  'line',
  2),
 (HPLP(column=None, hi_col='high', lo_col='low', span=10),
  [{'color': 'green', 'size': 10, 'opacity': 0.8},
   {'color': 'red', 'size': 10, 'opacity': 0.8}],
  'points',
  1),
 (ConsolidationZone(column=None, hp_column='HP_hi_10', lp_column='LP_lo_10', price_tolerance=0.001, max_points_between=1, height_width_ratio=50, name='RECT', atr_column='ATR_50'),
  [{'color': 'rgba(225, 182, 30, 0.5)',
    'fillcolour': 'rgba(225, 182, 30, 0.1)',
    'width': 2},
   {'color': 'rgba(225, 182, 30, 0.1)',
    'fillcolour': 'rgba(225, 182, 30, 0.1)',
    'width': 2}],
  'rect',
  1),
 (MicroTrendline(name='MTREND', pointsCol='HP_

In [21]:
def find_object_by_name(object_list, target_name, namesExtend=None):
    """
    Iterates through a list of tuples containing objects and returns the names
    of the object that matches the target_name.
    
    Args:
        object_list: List of tuples where first element is an object
        target_name: Name to search for
        
    Returns:
        Names of the matching object or None if not found
    """
    for obj_tuple in object_list:
        obj = obj_tuple[0]  # Get the object from the tuple
        if hasattr(obj, 'name') and obj.name == target_name:
            if namesExtend:
                return [n for n in obj.names if namesExtend in n]
            return obj.names
    return None

# Example usage:
names = find_object_by_name(f.ta, "MTREND", namesExtend='UP')
# names = find_object_by_name(f.ta, "MTREND", namesExtend=None)
names

[]

In [1]:

    """
    Here I have a class outline. Please complete the class. look at the dock string and comments to understand the intent of the class. 
    You must maintain the _compute_row method as this class inherits from the Signals class which uses a run method which calls the _compute_row  method for each row.
    
    The columns to be computed from will be defined by searching using lineColStartsWith variable. 
    Each column that starts with this variable will also have a number appended to it. eg MTREND_DOWN_1, MTREND_DOWN_2, ...
    The computed output will be a corresponding column with the same name as the input column but with the word touches appended to it.
    The output will be computed on from the perspective of the last row of the data frame. (the _compute_row method is called for each row of the data frame)
    _compute_row should retrun a series with a vlaue for each column that is being computed. Note that it is just return a row of the data frame.




'Here I have a class outline. Please complete the class look at the dock string hopefully it is clear enough. \nYou must maintain the compute row method as this class inherits from signals which uses a run method which calls this compute method for each row.\nFor this particular class it needs to count the number of touches of a line which can be a resistance or support line it can be a trend line or it can be the upper or lower bounds of consolidation zones\nThe the function of the line is not necessarily important but now that some lines will be horizontal meaning that they are maintaining the same price level and other lines such as trend lines will be on a slope which means every single bar has a different value\n\nTo add further complexity each line line has its own separate column within the pandas data frame. So even though I have multiple say trend lines or consolidation zones within my data frame they all have their own columns for each line \n  The way to Identify the columns

In [4]:

def daily_breakout(f):
    #£ gap score
    #£ has room to move
    #£ volume spike
    # prev period was consolidation
    # period prior to that was a pullback
    #£ MArket Relative strength
    #£ Sector Relative strength

SyntaxError: incomplete input (1224903109.py, line 8)

In [ ]:
def calculate_shock_value(pivot_value, prev_low, prev_close, current_open, atr):
    """
    Calculate shock value measuring the magnitude of retracement and subsequent gap relative to a pivot.
    - Greater retracement (lower prev_close) leads to higher shock value
    - Total move relative to ATR must also be significant
    """
    # Distance calculations
    dist1 = pivot_value - prev_low  # Total retracement range
    dist2 = pivot_value - prev_close  # How far it actually retraced
    
    if dist2 <= 0:  # Price didn't retrace below pivot
        return 0.0
        
    # Component 1: Shock ratio normalized to range 0-1
    # Higher ratio (deeper retracement) gives higher score
    shock_ratio = dist2 / dist1  # How much of the possible retracement was achieved
    shock_component = max(0, min(shock_ratio, 1))
    
    # Component 2: Move significance relative to ATR
    total_move = current_open - prev_low  # Total move from low to current open
    atr_component = max(0, min(total_move / (10 * atr), 1))
    
    # Final score: Both components must be significant
    return shock_component * atr_component * 100

current_open = 110
pivot_value = 100
prev_low = 90
prev_close = 99

atr = 10

calculate_shock_value(pivot_value, prev_low, prev_close, current_open, atr)

2.0000000000000004

In [ ]:
f.data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'SPY_cl', 'ATR_50', 'MA_cl_9',
       'MA_cl_21', 'MA_cl_28', 'MA_vo_10', 'HP_hi_10', 'LP_lo_10', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'MRSI_14', 'SigL_PBB'],
      dtype='object')

In [ ]:
f.data['SigL_RTM']

date
2024-01-01 00:00:00     NaN
2024-01-01 00:15:00     NaN
2024-01-01 00:30:00     NaN
2024-01-01 00:45:00     NaN
2024-01-01 01:00:00     NaN
                       ... 
2024-01-04 01:45:00     NaN
2024-01-04 02:00:00     NaN
2024-01-04 02:15:00     NaN
2024-01-04 02:30:00     NaN
2024-01-04 02:45:00    22.3
Name: SigL_RTM, Length: 300, dtype: float64

In [67]:
def score_ma50_movement(
    move_height: float,
    move_duration: int,
    height_weight: float = 30,
    duration_weight: float = 30,
    steepness_weight: float = 40,
    optimal_duration_min: int = 15,
    optimal_duration_max: int = 25,
    duration_penalty_short: float = 2,
    duration_penalty_long: float = 1
) -> dict:
    """
    Score MA50 movement based on height, duration, and steepness.
    
    Args:
        move_height: Price movement in points
        move_duration: Number of bars the move took
        height_weight: Maximum points for height (default 30)
        duration_weight: Maximum points for duration (default 30)
        steepness_weight: Maximum points for steepness (default 40)
        optimal_duration_min: Minimum optimal duration in bars (default 15)
        optimal_duration_max: Maximum optimal duration in bars (default 25)
        duration_penalty_short: Points deducted per bar below min (default 2)
        duration_penalty_long: Points deducted per bar above max (default 1)
    
    Returns:
        Dictionary containing scores and ratios
    """
    # Height score
    height_score = (move_height / 100) * height_weight
    
    # Duration score
    duration_score = duration_weight
    if move_duration < optimal_duration_min:
        duration_score -= (optimal_duration_min - move_duration) * duration_penalty_short
    elif move_duration > optimal_duration_max:
        duration_score -= (move_duration - optimal_duration_max) * duration_penalty_long
    duration_score = max(0, duration_score)
    
    # Steepness score
    ratio = move_height / move_duration
    if ratio > 2:
        steepness_score = steepness_weight
    elif ratio > 1:
        steepness_score = steepness_weight * 0.75
    elif ratio > 0.5:
        steepness_score = steepness_weight * 0.5
    else:
        steepness_score = steepness_weight * 0.25
        
    base_score = height_score + duration_score
    total_score = base_score + steepness_score
    
    return {
        'total_score': round(total_score),
        'base_score': round(base_score),
        'height_score': round(height_score),
        'duration_score': round(duration_score),
        'steepness_score': round(steepness_score),
        'ratio': round(ratio, 2)
    }

score_ma50_movement(500, 500)

{'total_score': 170,
 'base_score': 150,
 'height_score': 150,
 'duration_score': 0,
 'steepness_score': 20,
 'ratio': 1.0}